In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
models = ['cnn','tabular', 'four_blocks','five_minutes','lgbm']

In [ ]:
test_df= pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')

In [ ]:
if len(test_df) == 3:models = []
    

In [ ]:
for m in models:
    script_path = f'../input/optiverscripts/{m}_script.py'
    !python {script_path}

In [ ]:
def get_aggregated():
    res = []
    for m in models:
        try:
            df = pd.read_feather(f'{m}.feather')
            df = pd.merge(test_df, df, on='row_id', how='left')
            res.append(df.target.to_numpy())
        except Exception as e:
            print(e)
            print(m, 'FAILED')
    if res:
        res = np.stack(res)
        test_df['target']=res.mean(axis=0)
    else:
        test_df['target'] =0.0018
    return test_df

In [ ]:
test_df = get_aggregated()

In [ ]:
test_df[['row_id', 'target']].to_csv('submission.csv', index =False)
pd.read_csv('submission.csv').head()